In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 128
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'CUB1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(312,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=200000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=train_id)[0], replace=True)
        batch_features_negative[i] = batch_features[pick]    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(200), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(200), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)

training...
episode: 1000 loss tensor(1.4877, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.02460397708122683
gzsl: unseen=0.0054 , seen=0.0102 , h=0.0071
____________________________________________________________________________________________________
episode: 2000 loss tensor(1.2324, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.045837546343107514
gzsl: unseen=0.0115 , seen=0.0232 , h=0.0154
____________________________________________________________________________________________________
episode: 3000 loss tensor(1.1078, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.07718233906302663
gzsl: unseen=0.0266 , seen=0.0340 , h=0.0299
____________________________________________________________________________________________________
episode: 4000 loss tensor(1.1643, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.1435793731041456
gzsl: unseen=0.0499 , seen

episode: 31000 loss tensor(0.7072, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33771486349848334
gzsl: unseen=0.1223 , seen=0.2358 , h=0.1611
____________________________________________________________________________________________________
episode: 32000 loss tensor(0.6620, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3343444556791372
gzsl: unseen=0.1176 , seen=0.2358 , h=0.1570
____________________________________________________________________________________________________
episode: 33000 loss tensor(0.7165, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3356926188068756
gzsl: unseen=0.1203 , seen=0.2324 , h=0.1586
____________________________________________________________________________________________________
episode: 34000 loss tensor(0.6902, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33636670037074484
gzsl: unseen=0.1217 , seen=0.2347 , 

episode: 61000 loss tensor(0.8258, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34041118975396023
gzsl: unseen=0.1362 , seen=0.2517 , h=0.1767
____________________________________________________________________________________________________
episode: 62000 loss tensor(0.5772, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34277047522750254
gzsl: unseen=0.1416 , seen=0.2500 , h=0.1808
____________________________________________________________________________________________________
episode: 63000 loss tensor(0.7393, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33872598584428715
gzsl: unseen=0.1405 , seen=0.2585 , h=0.1821
____________________________________________________________________________________________________
episode: 64000 loss tensor(0.8202, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3390630266262218
gzsl: unseen=0.1436 , seen=0.2540 ,

episode: 91000 loss tensor(0.4166, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33636670037074484
gzsl: unseen=0.1598 , seen=0.2636 , h=0.1989
____________________________________________________________________________________________________
episode: 92000 loss tensor(0.6895, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33704078193461406
gzsl: unseen=0.1564 , seen=0.2608 , h=0.1955
____________________________________________________________________________________________________
episode: 93000 loss tensor(0.8715, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3373778227165487
gzsl: unseen=0.1517 , seen=0.2625 , h=0.1922
____________________________________________________________________________________________________
episode: 94000 loss tensor(0.7224, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3346814964610718
gzsl: unseen=0.1574 , seen=0.2591 , 

episode: 121000 loss tensor(0.9475, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.330974047859791
gzsl: unseen=0.1571 , seen=0.2636 , h=0.1968
____________________________________________________________________________________________________
episode: 122000 loss tensor(0.5456, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3343444556791372
gzsl: unseen=0.1645 , seen=0.2596 , h=0.2014
____________________________________________________________________________________________________
episode: 123000 loss tensor(0.7096, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33838894506235256
gzsl: unseen=0.1702 , seen=0.2551 , h=0.2042
____________________________________________________________________________________________________
episode: 124000 loss tensor(0.6703, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33367037411526795
gzsl: unseen=0.1598 , seen=0.2619

...

episode: 361000 loss tensor(0.5614, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3380519042804179
gzsl: unseen=0.1995 , seen=0.2426 , h=0.2190
____________________________________________________________________________________________________
episode: 362000 loss tensor(0.7501, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3414223120997641
gzsl: unseen=0.2100 , seen=0.2398 , h=0.2239
____________________________________________________________________________________________________
episode: 363000 loss tensor(0.6515, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3390630266262218
gzsl: unseen=0.2039 , seen=0.2409 , h=0.2209
____________________________________________________________________________________________________
episode: 364000 loss tensor(0.6832, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33704078193461406
gzsl: unseen=0.2032 , seen=0.2432

episode: 391000 loss tensor(0.8959, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34074823053589487
gzsl: unseen=0.2056 , seen=0.2409 , h=0.2219
____________________________________________________________________________________________________
episode: 392000 loss tensor(0.5530, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34041118975396023
gzsl: unseen=0.2002 , seen=0.2432 , h=0.2196
____________________________________________________________________________________________________
episode: 393000 loss tensor(0.9457, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34344455679137176
gzsl: unseen=0.2076 , seen=0.2426 , h=0.2238
____________________________________________________________________________________________________
episode: 394000 loss tensor(0.5556, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3417593528816987
gzsl: unseen=0.2073 , seen=0.24

episode: 421000 loss tensor(0.6209, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34041118975396023
gzsl: unseen=0.2066 , seen=0.2421 , h=0.2229
____________________________________________________________________________________________________
episode: 422000 loss tensor(0.8993, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3414223120997641
gzsl: unseen=0.2086 , seen=0.2409 , h=0.2236
____________________________________________________________________________________________________
episode: 423000 loss tensor(0.4808, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3400741489720256
gzsl: unseen=0.2049 , seen=0.2404 , h=0.2212
____________________________________________________________________________________________________
episode: 424000 loss tensor(0.6810, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3414223120997641
gzsl: unseen=0.2096 , seen=0.2381

episode: 451000 loss tensor(0.5902, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34108527131782945
gzsl: unseen=0.2083 , seen=0.2421 , h=0.2239
____________________________________________________________________________________________________
episode: 452000 loss tensor(0.5421, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.339737108190091
gzsl: unseen=0.2083 , seen=0.2387 , h=0.2224
____________________________________________________________________________________________________
episode: 453000 loss tensor(0.4601, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3380519042804179
gzsl: unseen=0.2066 , seen=0.2404 , h=0.2222
____________________________________________________________________________________________________
episode: 454000 loss tensor(0.6537, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3400741489720256
gzsl: unseen=0.2066 , seen=0.2392 

episode: 481000 loss tensor(0.3517, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33872598584428715
gzsl: unseen=0.2100 , seen=0.2375 , h=0.2229
____________________________________________________________________________________________________
episode: 482000 loss tensor(0.8349, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34041118975396023
gzsl: unseen=0.2083 , seen=0.2392 , h=0.2227
____________________________________________________________________________________________________
episode: 483000 loss tensor(0.7047, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.339737108190091
gzsl: unseen=0.2080 , seen=0.2398 , h=0.2227
____________________________________________________________________________________________________
episode: 484000 loss tensor(0.6437, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.339737108190091
gzsl: unseen=0.2049 , seen=0.2415 